## Neural Network Model

Use 1D-KNN

Note: This code have to be run after preprocessing.ipynb

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import numpy as np
import pandas as pd
import os
import pickle
from pathlib import Path
from keras import backend as K

Using TensorFlow backend.


In [15]:
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import GlobalAveragePooling1D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras import regularizers
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import class_weight
from keras.layers import LeakyReLU

In [3]:
trainset = pickle.load(open("./trainset.p",'rb'))
testset = pickle.load(open("./testset.p",'rb'))

In [29]:
USE_KFOLD = True
testset.groupby('stripPVC').count()

,R_interval_mean,R_interval_sd,age,annotation,gender,id,lead,p2p_mean,p2p_std,peak,strip,p2p_diff_form_norm
stripPVC,,,,,,,,,,,,
False,937,937,937,937,937,937,937,937,937,937,937,937
True,674,674,674,674,674,674,674,674,674,674,674,674


In [5]:
xtrain = trainset['strip']
ytrain = [i == True for i in trainset['stripPVC']]
xtest = testset['strip']
ytest = [i == True for i in testset['stripPVC']]

In [6]:
# convert

xtrain = np.stack(xtrain)
xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1], 1))
xtest = np.stack(xtest)
xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[1], 1))
xtrain.shape

(2864, 3600, 1)

### 1D-CNN

In [51]:
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())
def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())
def CNN_2(input_shape, blocks = 2, dropout = 0, regularization = 0.001, kernel_size = 16):  
    classifier = Sequential()
    classifier.add(Conv1D(filters = 100, kernel_size = kernel_size, activation = 'relu', input_shape = (input_shape, 1)))
    classifier.add(Dropout(dropout))
    classifier.add(Conv1D(filters = 160, kernel_size = kernel_size, activation = 'relu'))
    classifier.add(MaxPooling1D(pool_size = 2))
    classifier.add(LeakyReLU())
    classifier.add(LeakyReLU())
    classifier.add(Dropout(dropout))
    classifier.add(GlobalAveragePooling1D())
    classifier.add(Dense(units = 1, activation = 'sigmoid'))
    
 
    
    classifier.compile(optimizer = 'adam',
                       loss = 'binary_crossentropy',
                       metrics = [sensitivity,specificity,'accuracy'])
    return classifier

In [52]:
MODEL_NUMBER = '1'
MODEL = CNN_2
N_SPLITS = 5
BATCH_SIZE = 10
EPOCHS = 10 

In [53]:
params = { 'blocks': 5, 'dropout': 0, 'regularization': 0, 'kernel_size': 16, 'reduceLR': True}
classifier = MODEL(input_shape=np.shape(xtrain)[1], blocks = params['blocks'], dropout=params['dropout'], regularization=params['regularization'])
callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)]

# class_weights = class_weight.compute_class_weight('balanced',
#                                                   np.unique(ytrain),
#                                                   ytrain)
history = classifier.fit(xtrain,
                         ytrain,
                         batch_size = BATCH_SIZE,
                         epochs = EPOCHS,
                         validation_split=0.2,
#                          class_weight = class_weights,
                         callbacks = callbacks
                        )

Train on 2291 samples, validate on 573 samples
Epoch 1/10
2291/2291 [==============================] - 209s 91ms/step - loss: 0.6842 - sensitivity: 0.8799 - specificity: 0.1046 - acc: 0.5133 - val_loss: 0.6373 - val_sensitivity: 0.9599 - val_specificity: 0.0257 - val_acc: 0.7225
Epoch 2/10
2291/2291 [==============================] - 190s 83ms/step - loss: 0.6560 - sensitivity: 0.7494 - specificity: 0.3639 - acc: 0.5749 - val_loss: 0.7144 - val_sensitivity: 0.4075 - val_specificity: 0.3628 - val_acc: 0.4538
Epoch 3/10
2291/2291 [==============================] - 212s 92ms/step - loss: 0.6398 - sensitivity: 0.6668 - specificity: 0.5418 - acc: 0.6102 - val_loss: 0.5527 - val_sensitivity: 0.8290 - val_specificity: 0.2422 - val_acc: 0.7417
Epoch 4/10
2291/2291 [==============================] - 211s 92ms/step - loss: 0.6078 - sensitivity: 0.6937 - specificity: 0.6100 - acc: 0.6499 - val_loss: 0.5195 - val_sensitivity: 0.9350 - val_specificity: 0.0816 - val_acc: 0.7365
Epoch 5/10
2291/2291 

In [54]:
trainpred = classifier.predict_classes(xtrain)
testpred = classifier.predict_classes(xtest)

In [56]:
def metric(pred, real):
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import roc_auc_score
    ConfusionMatrix= pd.DataFrame(confusion_matrix(pred,real), columns=['True','False'], index=['True','False'])
    print(ConfusionMatrix)
    print('----------------')
    print('Precision:', ConfusionMatrix.iloc[0,0] / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,0]))
    print('Recall:', ConfusionMatrix.iloc[0,0] / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,1]))
    print('Accuracy:', (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,1]) / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,0] + ConfusionMatrix.iloc[0,1] + ConfusionMatrix.iloc[1,1]))
    print('AUC:', roc_auc_score(pred,real))

In [55]:
# metric([t[0] for t in testpred], ytest)


### Integrate with randomforest

In [ ]:
from sklearn import preprocessing

col = trainset.drop(['peak','annotation','strip','stripPVC','id','lead'],axis=1).columns.values
scale = preprocessing.StandardScaler()
scale.fit(trainset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1))
trainset['cnnProb'] = trainpred
testset['cnnProb'] = testpred
n_trainset = pd.DataFrame(scale.transform(trainset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1)),columns=col)
n_testset = pd.DataFrame(scale.transform(testset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1)),columns=col)
